In [ ]:
!nvidia-smi

Tue Apr 11 00:46:09 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
!git clone https://github.com/hmartiro/riffusion-inference
%cd riffusion-inference

fatal: destination path 'riffusion-inference' already exists and is not an empty directory.
/content/riffusion-inference


In [ ]:
# Riffusion dependencies
!pip install accelerate
!pip install argh
!pip install dacite
!pip install demucs
!pip install diffusers>=0.9.0
!pip install flask
!pip install flask_cors
!pip install numpy
!pip install pillow==9.1.0
!pip install plotly
!pip install pydub
!pip install pysoundfile
!pip install scipy
!pip install soundfile
!pip install sox
!pip install streamlit>=1.10.0
!pip install torch
!pip install torchaudio
!pip install torchvision
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in

In [ ]:
from pathlib import Path
from typing import Union, Optional
from os import getcwd, sep
import io
import sys

import PIL

sys.path.append(getcwd() + sep + "riffusion")

from riffusion.datatypes import InferenceInput, InferenceOutput
from riffusion.riffusion_pipeline import RiffusionPipeline
from riffusion.spectrogram_image_converter import SpectrogramImageConverter
from riffusion.spectrogram_params import SpectrogramParams
from riffusion.server import SEED_IMAGES_DIR



pipeline: RiffusionPipeline


def startup(
        checkpoint: str = "riffusion/riffusion-model-v1",
        no_traced_unet: bool = False,
        device: str = "cuda",
            ) -> None:
    """
    Initializes the pipeline.
    
    Parameters:
        checkpoint (str): a string of where to load a pretrained
        model.
        no_traced_unet (bool): whether to not use a traced unet
        for speedups.
        device (str): "cuda", "cpu", or "mps".

    Returns:
        None
    """
    global pipeline
    pipeline = RiffusionPipeline.load_checkpoint(
        checkpoint=checkpoint,
        use_traced_unet=not no_traced_unet,
        device=device,
    )

def compute(inputs: InferenceInput) -> InferenceOutput:
    """
    Function from the riffusion server :func:`compute_request`.
    
    Parameters:
        inputs (:py:class:`InferenceInput`): The inputs for the request.
    
    Returns:
        str: an :py:class:`InferenceOutput`.
    """
    # Load the seed image by ID
    init_image_path = Path(SEED_IMAGES_DIR, f"{inputs.seed_image_id}.png")

    if not init_image_path.is_file():
        return f"Invalid seed image: {inputs.seed_image_id}", 400
    init_image = PIL.Image.open(str(init_image_path)).convert("RGB")

    # Load the mask image by ID
    mask_image: Optional[PIL.Image.Image] = None
    if inputs.mask_image_id:
        mask_image_path = Path(SEED_IMAGES_DIR, f"{inputs.mask_image_id}.png")
        if not mask_image_path.is_file():
            return f"Invalid mask image: {inputs.mask_image_id}", 400
        mask_image = PIL.Image.open(str(mask_image_path)).convert("RGB")

    # Execute the model to get the spectrogram image
    image = pipeline.riffuse(
        inputs,
        init_image=init_image,
        mask_image=mask_image,
    )

    # TODO(hayk): Change the frequency range to [20, 20k] once the model is retrained
    params = SpectrogramParams(
        min_frequency=0,
        max_frequency=10000,
    )

    # Reconstruct audio from the image
    # TODO(hayk): It may help performance a bit to cache this object
    converter = SpectrogramImageConverter(params=params, device=str(pipeline.device))

    segment = converter.audio_from_spectrogram_image(
        image,
        apply_filters=True,
    )
    return segment

In [ ]:
startup(device="cuda")

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

You have passed a non-standard module <function RiffusionPipeline.load_checkpoint.<locals>.<lambda> at 0x7fac64c0c550>. We cannot verify whether it has the correct type
/usr/local/lib/python3.9/dist-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(


In [ ]:
import base64
import json
from riffusion.datatypes import InferenceInput, PromptInput, InferenceOutput

seed: int = 1

def muse(prompt_start_input, prompt_end_input, alpha_input, num_steps: int = 50):    
    segment = museHelp(prompt_start_input, prompt_end_input, alpha_input, num_steps)

    # Export audio to MP3 bytes
    wav_bytes = io.BytesIO()
    segment.export(wav_bytes, format="wav")
    wav_bytes.seek(0)

    with open("output.mp3", "wb") as f:
        f.write(wav_bytes)
    return "output.mp3"

def museHelp(prompt_start_input, prompt_end_input, alpha_input, num_steps: int = 50):
    global seed
    transition: bool = prompt_start_input != prompt_end_input
    promptStart: PromptInput = PromptInput(
        prompt=prompt_start_input,
        seed=seed,
    )
    seed += 1 if transition else 0
    promptEnd: PromptInput = PromptInput(
        prompt=prompt_end_input,
        seed=seed,
    )
    inputTotal: InferenceInput = InferenceInput(
        start=promptStart,
        end=promptEnd,
        alpha=alpha_input,
        num_inference_steps=num_steps,
    )
    
    segment = compute(inputs=inputTotal)
    return segment


In [ ]:
!pip install sounddevice

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import sounddevice as sd

In [ ]:
from IPython.display import Audio
from IPython.display import display

promptStart = "classical piano" # @param {type: "string"}
promptEnd = "heavy metal" # @param {type: "string"}
alpha = 0 # @param {type:"slider", min:0.0, max:1, step:0.01}

segment  = museHelp(promptStart, promptEnd, alpha)
#filename = muse(promptStart, promptEnd, alpha)
wn = Audio(data=segment.get_array_of_samples(), rate=segment.frame_rate, autoplay=True)
display(wn)

  0%|          | 0/38 [00:00<?, ?it/s]

In [ ]:
import socket, os
import pydub

server_ip: str = '192.168.1.102' # @param {type: "string"}
port: int = 9611 # @param {type: "number"}
CHUNK: int = 1024
sample_rate: int = 44100
client_socket = socket.socket(socket.AF_INET,socket.SOCK_STREAM)
socket_address = (server_ip,port)
client_socket.connect(socket_address)
client: dict = {"client": "music", "threads": "1"}
client_socket.sendall(str(client))

In [ ]:
def sendAudio(segment: pydub.AudioSegment):
  data = segment.get_array_of_samples()
  for i in range(0, len(data) + CHUNK, CHUNK):
    mini = data[i:min(i + CHUNK, len(data) - 1)]
    client_socket.sendall(mini)

In [ ]:
import ast

while True:
    message = client_socket.recv(50)
    try:
      inputs = ast.literal_eval(message)
      sendAudio(museHelp(inputs["promptStart"], inputs["promptEnd"], inputs["alpha"]))
    except ValueError e:
      print(e)